In [1]:
# Import libraries
import numpy as np
import pandas as pd
import string
from datetime import datetime

In [34]:
# Define the input files
file_ref = '../rawdata/snippets_reference_K23.csv'
file_user_raw = '../rawdata/snippets_luval_pre_AK23.csv'
file_output1 = 'output/snippets_output_ratios_pre_AK23'   # Can be csv or xlsx
file_output2 = 'output/snippets_output_stats_pre_AK23'   # Can be csv or xlsx

# Define the relevent texts
max_num_rel_text = 3
dict_rel_text = {'S': [148, 149, 150], 
                 'T': [151, 152, 153]}

In [3]:
# Read the reference snippets
df_ref = pd.read_csv(file_ref, index_col=0)
df_ref

,text_id,snippet_text,words
sid,,,
0,148,auringon lämpöenergiaa tarvitaan,3
1,148,vedellä on kolme olomuotoa,4
2,148,kiinteä nestemäinen ja kaasumainen,4
3,148,lämpötila vaikuttaa veden olomuotoon,4
4,148,veden olomuodon muutos,3
5,148,aurinko lämmittää maapallon pintaa,4
6,148,lämpöenergia saa veden haihtumaan,4
7,148,muuttamaan olomuotoa nesteestä vesihöyryksi,4
8,148,eri ympäristöissä haihtumista tapahtuu eri tav...,6


In [4]:
# Read the raw user snippets
df_user_raw = pd.read_csv(file_user_raw, sep=';')
df_user_raw

,anonym,instance_id,text_id,snippet_text,ordernumber,group_id,group_title,created,deactivated
0,AK23S0001,PELAS,150.0,Pienet vesipisarat törmäilevät ja yhdistyvät v...,5,NaN,NaN,2023-02-07 07:43:23.454+00,2023-02-07 07:45:22.885923+00
1,AK23S0001,PELAS,150.0,"nostamaan pisaroita ylöspäin, kunnes pisarat o...",3,7.0,Sade,2023-02-07 07:43:33.818+00,2023-02-07 08:06:18.956801+00
2,AK23S0001,PELAS,150.0,Pilven sisällä vesipisarat ovat jatkuvassa lii...,0,7.0,Sade,2023-02-07 07:47:49.438+00,2023-02-07 08:06:11.165957+00
3,AK23S0001,PELAS,150.0,pilvet koostuvat vesihöyrystä,2,7.0,Sade,2023-02-07 07:50:33.948+00,NaN
4,AK23S0001,PELAS,150.0,Vesipisarat tai jääkiteet ovat niin pieniä ja ...,1,7.0,Sade,2023-02-07 07:51:14.826+00,NaN
...,...,...,...,...,...,...,...,...,...
1913,AK23S8287,SOSKS,148.0,"Lämpöenergia saa veden haihtumaan, eli muuttam...",4,NaN,NaN,2023-01-27 07:21:07.875+00,NaN
1914,AK23S8287,SOSKS,149.0,"Kun ilma kohoaa ja sen myötä jäähtyy, saavutet...",3,NaN,NaN,2023-01-27 07:29:43.034+00,NaN
1915,AK23S8287,SOSKS,149.0,"Kun ilma kohoaa ja sen myötä jäähtyy, saavutet...",2,NaN,NaN,2023-01-27 07:29:44.003+00,NaN
1916,AK23S8287,SOSKS,150.0,Vesipisarat tai jääkiteet ovat niin pieniä ja ...,1,NaN,NaN,2023-01-27 07:52:04.785+00,NaN


In [20]:
# Data Preprocessing for User Snippets
# (1) These rows are removed at this moment:
#    (a) Deactivated rows, which are snippets that have been unmarked;
#    (b) NaN in text_id, which are self-inputted snippets;
#    (c) Exactly repeated snippets.
# (2) Rename the index as 'usid' (user snippets ID) afterwards

df_user = df_user_raw[df_user_raw["deactivated"].isna()].copy()
df_user = df_user[~df_user["text_id"].isna()]
df_user = df_user.drop_duplicates(
    subset=["anonym", "snippet_text"]
).reset_index(drop=True)
df_user.index.name = "usid"
df_user['text_id'] = df_user['text_id'].astype(int)
df_user


,anonym,instance_id,text_id,snippet_text,ordernumber,group_id,group_title,created,deactivated
usid,,,,,,,,,
0,AK23S0001,PELAS,150,pilvet koostuvat vesihöyrystä,2,7.0,Sade,2023-02-07 07:50:33.948+00,NaN
1,AK23S0001,PELAS,150,Vesipisarat tai jääkiteet ovat niin pieniä ja ...,1,7.0,Sade,2023-02-07 07:51:14.826+00,NaN
2,AK23S0002,PELAS,150,Pilvet muodostuvat vesipisaroista tai jääkitei...,19,NaN,NaN,2023-02-07 07:55:18.065+00,NaN
3,AK23S0003,PELAS,150,"Arkikielessä sanomme usein, että pilvet koost...",2,NaN,NaN,2023-02-07 07:47:24.446+00,NaN
4,AK23S0003,PELAS,150,Vesipisarat tai jääkiteet ovat niin pieniä ja ...,1,NaN,NaN,2023-02-07 07:48:35.329+00,NaN
...,...,...,...,...,...,...,...,...,...
1220,AK23S8252,SKMTS,150,Pilven sisällä vesipisarat ovat jatkuvassa lii...,0,7.0,tiatgoteksti,2023-02-02 11:13:26.834+00,NaN
1221,AK23S8287,SOSKS,148,"Lämpöenergia saa veden haihtumaan, eli muuttam...",4,NaN,NaN,2023-01-27 07:21:07.875+00,NaN
1222,AK23S8287,SOSKS,149,"Kun ilma kohoaa ja sen myötä jäähtyy, saavutet...",3,NaN,NaN,2023-01-27 07:29:43.034+00,NaN


In [6]:
# Define a function for finding the max consecutive common subsequence
def longest_consecutive_common_subsequence(a, b):
    max_length = 0
    end_index = -1
    dp = [[0 for _ in range(len(b) + 1)] for _ in range(len(a) + 1)]

    for i in range(1, len(a) + 1):
        for j in range(1, len(b) + 1):
            if a[i - 1] == b[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                if dp[i][j] > max_length:
                    max_length = dp[i][j]
                    end_index = i - 1
            else:
                dp[i][j] = 0

    if max_length == 0:
        return []

    return a[end_index - max_length + 1 : end_index + 1]

### Evaluation for each user snippets

Two properties are defined for each user snippets: **Relevant Proportion** and **Coverage Ratio**. 

**Relevant Proportion**: This ratio measures how many words from the reference snippet are matched or found within the user snippet, indicating the accuracy or precision of the match. A minimum of three consecutive words must match for inclusion in the calculation.

**Coverage Proportion**: This ratio represents the proportion of words in the user snippet relative to the reference snippet, indicating how much of the reference content the user snippet covers.

When a user snippet matches more than one reference snippet, only the reference snippet with the longest continuous match of words will be considered.

For example:
```
Reference snippet: "There are many forests in Finland."
User snippet: "forests in Finland. The"
```
The relevant proportion is 3/4.
The coverage proportion is 3/6.

In [21]:
# A nested loop: loop over all User Snippets (usid), then all Reference Snippets (rsid)
# Two new columns named 'relevant_proportion' and 'coverage_proportion' will be created in the User Snippets DataFrame

for usid, urow in df_user.iterrows():

    # if a reference snippet is found in the user snippet, turn those words to "True"
    urow_clean = (
        urow["snippet_text"]
        .strip()
        .lower()
        .translate(str.maketrans("", "", string.punctuation))
    )

    # Initialize a list of "False" values with the same length as uwords
    uwords = urow_clean.split()
    uwords_status = [False] * len(uwords)
    max_num_common = 0

    # Find the reference snippet with the longest continuous match of words 
    for rsid, rrow in df_ref.iterrows():
        if rrow["text_id"] == urow["text_id"] and not pd.isna(urow["snippet_text"]):
            rwords = rrow["snippet_text"].split()
            common_subsequence = longest_consecutive_common_subsequence(rwords, uwords)
            num_common = len(common_subsequence)
            if num_common >= 3 and num_common > max_num_common:
                max_rsid = rsid
                max_num_common = num_common
                max_start_index = uwords.index(common_subsequence[0])
                num_refernce_snippets_words = len(rrow["snippet_text"].split())

    if (max_num_common >= 3):
        num_user_snippets_words = len(urow["snippet_text"].split())
        df_user.at[usid, "relevant_rsid"] = max_rsid
        df_user.at[usid, "relevant_proportion"] = max_num_common / num_user_snippets_words
        df_user.at[usid, "coverage_proportion"] = max_num_common / num_refernce_snippets_words

# Preview the result
df_user

,anonym,instance_id,text_id,snippet_text,ordernumber,group_id,group_title,created,deactivated,relevant_rsid,relevant_proportion,coverage_proportion
usid,,,,,,,,,,,,
0,AK23S0001,PELAS,150,pilvet koostuvat vesihöyrystä,2,7.0,Sade,2023-02-07 07:50:33.948+00,NaN,NaN,NaN,NaN
1,AK23S0001,PELAS,150,Vesipisarat tai jääkiteet ovat niin pieniä ja ...,1,7.0,Sade,2023-02-07 07:51:14.826+00,NaN,NaN,NaN,NaN
2,AK23S0002,PELAS,150,Pilvet muodostuvat vesipisaroista tai jääkitei...,19,NaN,NaN,2023-02-07 07:55:18.065+00,NaN,18.0,0.172414,1.0
3,AK23S0003,PELAS,150,"Arkikielessä sanomme usein, että pilvet koost...",2,NaN,NaN,2023-02-07 07:47:24.446+00,NaN,NaN,NaN,NaN
4,AK23S0003,PELAS,150,Vesipisarat tai jääkiteet ovat niin pieniä ja ...,1,NaN,NaN,2023-02-07 07:48:35.329+00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1220,AK23S8252,SKMTS,150,Pilven sisällä vesipisarat ovat jatkuvassa lii...,0,7.0,tiatgoteksti,2023-02-02 11:13:26.834+00,NaN,19.0,0.258065,1.0
1221,AK23S8287,SOSKS,148,"Lämpöenergia saa veden haihtumaan, eli muuttam...",4,NaN,NaN,2023-01-27 07:21:07.875+00,NaN,6.0,0.181818,1.0
1222,AK23S8287,SOSKS,149,"Kun ilma kohoaa ja sen myötä jäähtyy, saavutet...",3,NaN,NaN,2023-01-27 07:29:43.034+00,NaN,15.0,0.125000,1.0


In [27]:
df_user_encoded = pd.get_dummies(df_user, columns=['text_id'], prefix='text_id')
df_user_encoded[50:100]

,anonym,instance_id,snippet_text,ordernumber,group_id,group_title,created,deactivated,relevant_rsid,relevant_proportion,...,text_id_141,text_id_142,text_id_144,text_id_146,text_id_148,text_id_149,text_id_150,text_id_151,text_id_152,text_id_153
usid,,,,,,,,,,,,,,,,,,,,,
50,AK23S0015,PELAT,Pallo liikkuu samaan suuntaan ja samalla nopeu...,0,1.0,kuumailmapallolennot,2023-02-07 07:47:01.322+00,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
51,AK23S0015,PELAT,Sumuja ennustettaessa meteorologi seuraa esime...,0,3.0,Sumu,2023-02-07 07:53:01.594+00,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
52,AK23S0015,PELAT,Myrsky voidaan useissa tapauksissa korvata esi...,0,2.0,säätermit,2023-02-07 07:53:45.233+00,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
53,AK23S0016,PELAT,Aurinko lämmittää maapallon pintaa lämpöenergi...,0,2.0,Tuuli,2023-02-07 07:39:35.454+00,NaN,37.0,0.192308,...,False,False,False,False,False,False,False,False,True,False
54,AK23S0016,PELAT,"Tuuli, jonka tunnemme suoraan kasvoillamme, on...",2,2.0,Tuuli,2023-02-07 07:42:59.318+00,NaN,25.0,0.136364,...,False,False,False,False,False,False,False,True,False,False
55,AK23S0016,PELAT,"Kun ilma kohoaa riittävän ylös, se alkaa jälle...",1,2.0,Tuuli,2023-02-07 07:44:52.183+00,NaN,28.0,0.333333,...,False,False,False,False,False,False,False,True,False,False
56,AK23S0016,PELAT,"Tuulityypit riippuvat tuulen voimakkuudesta, s...",3,2.0,Tuuli,2023-02-07 07:48:41.476+00,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,True
57,AK23S0017,PELAT,Ilman lämpötilaerot ovat osa tuulen syntyä,2,NaN,NaN,2023-02-07 07:52:17.32+00,NaN,NaN,NaN,...,False,False,False,False,False,False,False,True,False,False
58,AK23S0017,PELAT,Ilma liikkuu ympärillämme koko ajan lämpötilae...,0,NaN,NaN,2023-02-07 07:57:49.094+00,NaN,24.0,0.388889,...,False,False,False,False,False,False,False,True,False,False


I have this df

	anonym	instance_id	snippet_text	ordernumber	group_id	group_title	created	deactivated	relevant_rsid	relevant_proportion	...	text_id_141	text_id_142	text_id_144	text_id_146	text_id_148	text_id_149	text_id_150	text_id_151	text_id_152	text_id_153
usid																					
0	AK23S0001	PELAS	pilvet koostuvat vesihöyrystä	2	7.0	Sade	2023-02-07 07:50:33.948+00	NaN	NaN	NaN	...	False	False	False	False	False	False	True	False	False	False
1	AK23S0001	PELAS	Vesipisarat tai jääkiteet ovat niin pieniä ja ...	1	7.0	Sade	2023-02-07 07:51:14.826+00	NaN	NaN	NaN	...	False	False	False	False	False	False	True	False	False	False
2	AK23S0002	PELAS	Pilvet muodostuvat vesipisaroista tai jääkitei...	19	NaN	NaN	2023-02-07 07:55:18.065+00	NaN	18.0	0.172414	...	False	False	False	False	False	False	True	False	False	False
3	AK23S0003	PELAS	Arkikielessä sanomme usein, että pilvet koost...	2	NaN	NaN	2023-02-07 07:47:24.446+00	NaN	NaN	NaN	...	False	False	False	False	False	False	True	False	False	False
4	AK23S0003	PELAS	Vesipisarat tai jääkiteet ovat niin pieniä ja ...	1	NaN	NaN	2023-02-07 07:48:35.329+00	NaN	NaN	NaN	...	False	False	False	False	False	False	True	False	False	False
...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...
1220	AK23S8252	SKMTS	Pilven sisällä vesipisarat ovat jatkuvassa lii...	0	7.0	tiatgoteksti	2023-02-02 11:13:26.834+00	NaN	19.0	0.258065	...	False	False	False	False	False	False	True	False	False	False
1221	AK23S8287	SOSKS	Lämpöenergia saa veden haihtumaan, eli muuttam...	4	NaN	NaN	2023-01-27 07:21:07.875+00	NaN	6.0	0.181818	...	False	False	False	False	True	False	False	False	False	False
1222	AK23S8287	SOSKS	Kun ilma kohoaa ja sen myötä jäähtyy, saavutet...	3	NaN	NaN	2023-01-27 07:29:43.034+00	NaN	15.0	0.125000	...	False	False	False	False	False	True	False	False	False	False

I want to create a new df_stat, such that each row represents a unique anonym with the following coloumns:
1. anonym: the unique anonym
2. snippets_total: number of rows of the same anonym in df
3. sum_text_id_###: how many rows of text_id_150 (for example) there for the same anonym
4. mean_relevant_proportion: mean value of relevant_proportion, excluding those NaN value

Do not need to include the above df in your code.

In [46]:
# Group by 'anonym' and aggregate data
df_stat = df_user_encoded.groupby('anonym').agg({
    'instance_id': lambda x: x.iloc[-1][-1],  # Get the last character of the last 'instance_id'
    'snippet_text': 'count',                  # Count the number of snippets per anonym
    'relevant_proportion': 'mean',            # Calculate mean of relevant_proportion, automatically excludes NaN
    'coverage_proportion': 'mean'             # Calculate mean of coverage_proportion, automatically excludes NaN
}).rename(columns={
    'instance_id': 'task',
    'snippet_text': 'snippets_total',
    'relevant_proportion': 'mean_relevant_proportion',
    'coverage_proportion': 'mean_coverage_proportion'
})

# creating a placeholder for relevant texts
df_stat['relevant_snippets_total'] = 0
for i in range(1,max_num_rel_text+1):
    df_stat['sum_relevant_text_'+str(i)] = 0

# For each text_id_### column, sum the True values (assuming these columns are boolean as per your description)
text_id_cols = [col for col in df_user_encoded if col.startswith('text_id_')]
for col in text_id_cols:
    df_stat['sum_' + col] = df_user_encoded.groupby('anonym')[col].sum()

# !!! HUOM !!!
# This part is task-specific! Refer to the definition of dict_rel_text at the beginning
def get_sum_relevant_text(row, i):
    task = row['task']
    text_id = dict_rel_text[task][i]
    return row['sum_text_id_'+str(text_id)]

for i in range(max_num_rel_text):
    df_stat['sum_relevant_text_'+str(i+1)] = df_stat.apply(get_sum_relevant_text, axis=1, args=(i,))
    df_stat['relevant_snippets_total'] += df_stat['sum_relevant_text_'+str(i+1)]

# Reset index to make 'anonym' a column again if needed
df_stat.reset_index(inplace=True)

df_stat

,anonym,task,snippets_total,mean_relevant_proportion,mean_coverage_proportion,relevant_snippets_total,sum_relevant_text_1,sum_relevant_text_2,sum_relevant_text_3,sum_text_id_113,...,sum_text_id_141,sum_text_id_142,sum_text_id_144,sum_text_id_146,sum_text_id_148,sum_text_id_149,sum_text_id_150,sum_text_id_151,sum_text_id_152,sum_text_id_153
0,AK23S0001,S,2,NaN,NaN,2,0,0,2,0,...,0,0,0,0,0,0,2,0,0,0
1,AK23S0002,S,1,0.172414,1.000000,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,AK23S0003,S,3,1.000000,1.000000,3,0,0,3,0,...,0,0,0,0,0,0,3,0,0,0
3,AK23S0004,S,12,0.733894,1.000000,11,1,0,10,1,...,0,0,0,0,1,0,10,0,0,0
4,AK23S0005,S,9,0.438508,1.000000,9,0,4,5,0,...,0,0,0,0,0,4,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,AK23S8208,T,4,0.090909,1.000000,4,1,3,0,0,...,0,0,0,0,0,0,0,1,3,0
244,AK23S8210,T,2,0.388889,0.541667,2,2,0,0,0,...,0,0,0,0,0,0,0,2,0,0
245,AK23S8212,T,3,0.150000,1.000000,3,0,3,0,0,...,0,0,0,0,0,0,0,0,3,0
246,AK23S8252,S,2,0.258065,1.000000,1,0,0,1,1,...,0,0,0,0,0,0,1,0,0,0


In [47]:
# Export the results to Excel
table1 = df_user[['anonym','snippet_text','relevant_rsid','relevant_proportion','coverage_proportion']]
table1.to_excel(file_output1+'.xlsx')

table2 = df_stat
table2.to_excel(file_output2+'.xlsx')